# BingSearch using Kernel 

This notebook explains how to integrate Bing Search with the Semantic Kernel  to get the latest information from the internet.

To use Bing Search you simply need a Bing Search API key. You can get the API key by creating a [Bing Search resource](https://learn.microsoft.com/en-us/bing/search-apis/bing-web-search/create-bing-search-service-resource) in Azure. 

To learn more read the following [documentation](https://learn.microsoft.com/en-us/bing/search-apis/bing-web-search/overview).


Prepare a Semantic Kernel instance first, loading also the AI backend settings defined in the [Setup notebook](0-AI-settings.ipynb):

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.0.0-beta6"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.0.0-beta6"

#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Plugins.Core;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.Planning;
using Microsoft.SemanticKernel.Planners;
using Microsoft.SemanticKernel.TemplateEngine;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

var builder = new KernelBuilder();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.WithAzureOpenAIChatCompletionService(model, azureEndpoint, apiKey);
else
    builder.WithOpenAIChatCompletionService(model, apiKey, orgId);

var kernel = builder.Build();

Installed Packages Microsoft.SemanticKernel, 1.0.0-beta6 Microsoft.SemanticKernel.Plugins.Web, 1.0.0-beta6

Add the following namespaces to use the Bing Search connector.

In [2]:
using Microsoft.SemanticKernel.Plugins.Web;
using Microsoft.SemanticKernel.Plugins.Web.Bing;

Enter your Bing Search Key in BING_KEY using `InteractiveKernel` method as introduced in [`.NET Interactive`](https://github.com/dotnet/interactive/blob/main/docs/kernels-overview.md)

In [3]:
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

string BING_KEY = (await InteractiveKernel.GetPasswordAsync("Please enter your Bing Search Key")).GetClearTextPassword();

Below are some examples on how [`WebSearchEnginePlugin`](../src/Plugins/Plugins.Web/WebSearchEnginePlugin.cs) can be used in SK. 

In [5]:
private static async Task Example1Async(IKernel kernel)
{
        Console.WriteLine("Example 1");

        // Run 
        var question = "What is quantum tunnelling";
        var function = kernel.Functions.GetFunction("bing", "search");
        var bingResult = await kernel.RunAsync(question, function);
        var bingResultString = bingResult.GetValue<string>();

        Console.WriteLine(question);
        Console.WriteLine("----");
        Console.WriteLine(bingResultString);
        Console.WriteLine();

        /* OUTPUT:

            What is quantum tunnelling
            ----
            In physics, quantum tunnelling, barrier penetration, or simply tunnelling is a quantum mechanical
            phenomenon in which an object such as an electron or atom passes through a potential energy 
            barrier that, according to classical mechanics, the object does not have sufficient energy to
            enter or surmount.
            
       */
}

In [6]:
private static async Task Example2Async(IKernel kernel)
{
    Console.WriteLine("Example 2");

    //The following function only works in interactive notebooks
    string question = await InteractiveKernel.GetInputAsync("Please ask your question"); 

    var function = kernel.Functions.GetFunction("bing", "search");
    var bingResult = await kernel.RunAsync(question, function);
    var bingResultString = bingResult.GetValue<string>();

    Console.WriteLine(bingResultString);
}

Just uncomment the examples to run them in the following code:

In [8]:
// Load Bing plugin
var bingConnector = new BingConnector(BING_KEY);

kernel.ImportFunctions(new WebSearchEnginePlugin(bingConnector), "bing");

//await Example1Async(kernel);
//await Example2Async(kernel);

Example 2
["AI has the power to make massive improvements to our quality of life, but it\u2019s not perfect. Artificial intelligence, or AI, is everywhere right now. In truth, the fundamentals of AI and...","Artificial intelligence (AI) technology is developing at high speed, transforming many aspects of modern life. However, some experts fear it could be used for malicious purposes. The UK is...","Artificial Intelligence With AI adoption on the rise, the technology is addressing a number of global challenges. Artificial intelligence is being used to help crack down on illegal activities. It is helping to tackle overfishing in our ocean. And it is even being used to address the issue of gender imbalance.","AI can be used to enhance the accuracy and efficiency of decision-making and to improve lives through new apps and services. It can be used to solve some of the thorny policy problems of climate change, infrastructure and healthcare.","Today, AI empowers organizations, governments an